In [3]:
import os 
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import (Input, Dense, Dropout, GlobalAveragePooling2D, RandomFlip,RandomRotation, RandomZoom, RandomTranslation)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, ReduceLROnPlateau)
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix,classification_report
import itertools

In [4]:
data_dir= "train"
IMG_SIZE=(224,224)
Batch_Size=32
SEED=42
train_ds= image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    batch_size=Batch_Size,
    image_size=IMG_SIZE,
    seed= SEED
)
val_ds= image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    batch_size=Batch_Size,
    image_size=IMG_SIZE,
    seed= SEED
)
class_names= train_ds.class_names
print("Classes:",class_names)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [5]:
AUTOTUNE= tf.data.AUTOTUNE
train_ds= train_ds.shuffle(1000).prefetch(AUTOTUNE)
val_ds= val_ds.prefetch(AUTOTUNE)

In [6]:
data_augmentation= tf.keras.Sequential([
    RandomFlip('horizontal'),
    RandomRotation(0.07),
    RandomZoom(0.1),
    RandomTranslation(0.05,0.05),
],name="data_augmentation")

In [10]:
base_model= EfficientNetB0(include_top=False,weights='imagenet',input_shape=(IMG_SIZE[0],IMG_SIZE[1],3))
base_model.trainable= False

inputs= Input(shape=(IMG_SIZE[0],IMG_SIZE[1],3))
x=data_augmentation(inputs)
x=preprocess_input(x)
x=base_model(x,training=False)
x=GlobalAveragePooling2D()(x)
x=Dropout(0.35)(x)
x=Dense(256,activation='relu')(x)
x=Dropout(0.3)(x)
outputs=Dense(len(class_names),activation='softmax')(x)

model=Model(inputs,outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ data_augmentation (Sequential)       │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,379,306 (16.71 MB)

 Trainable params: 329,735 (1.26 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(3e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [14]:
checkpoint_cb = ModelCheckpoint(
    "best_effnet.h5", save_best_only=True,
    monitor="val_accuracy", mode="max"
)

earlystop_cb = EarlyStopping(
    monitor="val_loss", patience=6,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5,
    patience=3, min_lr=1e-6
)
